**mnist-神经网络**

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)#读取数据
#mnist.train.images是一个55000 * 784维的矩阵, mnist.train.labels是一个55000 * 10维的矩阵

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
trX,trY,teX,teY = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels
print(trX.shape)
print(trY.shape)

(55000, 784)
(55000, 10)


In [14]:
print(teX.shape)

(10000, 28, 28, 1)


In [4]:
trX = trX.reshape(-1,28,28,1)
teX = teX.reshape(-1,28,28,1)

In [5]:
# INPUTS AND OUTPUTS
x = tf.placeholder(tf.float32, [None, 28,28,1]) # 用placeholder先占地方，样本个数不确定为None
y = tf.placeholder(tf.float32, [None,10 ]) # 用placeholder先占地方，样本个数不确定为None
p_keep_conv = tf.placeholder(tf.float32) #dopout参数
p_keep_hidden = tf.placeholder(tf.float32) #dropout参数

In [6]:
# w1=tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=0.1))
# w2=tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=0.1))
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.1))
regularizer = tf.contrib.layers.l2_regularizer(scale=1.0 / 5000)

w4 = tf.get_variable(
        name="w4",
        regularizer=regularizer,
        initializer=tf.random_normal([128*4*4, 625], stddev=0.1))

weights = {
    'w1': init_weights([3,3,1,32]),
    'w2': init_weights([3,3,32,64]),
    'w3': init_weights([3,3,64,128]),
    
    
    
    'w4': w4,#按照高斯分布初始化权重
    'out': init_weights([625,10])
}
biases = {
    'b1':tf.Variable(tf.constant(0.1,shape=[32])),
    'b2':tf.Variable(tf.constant(0.1,shape=[64])),
    'b3':tf.Variable(tf.constant(0.1,shape=[128])),
    'b4': tf.Variable(tf.constant(0.1,shape=[625])),
    'out': tf.Variable(tf.constant(0.1,shape=[10]))
}
print("NETWORK READY")


NETWORK READY


**加入L2正则**

In [7]:
tf.add_to_collection(tf.GraphKeys.WEIGHTS,w4)
# regularizer = tf.contrib.layers.l2_regularizer(scale=1.0 / 50000)
# print(regularizer)
reg_term = tf.contrib.layers.apply_regularization(regularizer)

# loss = (tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=z_3)) +
#         reg_term)

In [8]:
def model (X,weights,biases,p_keep_conv,p_keep_hidden):
    #第一组卷积层及池化层，最后dropout部分神经元
    l1a = tf.nn.selu(tf.nn.conv2d(X,weights['w1'],strides=[1,1,1,1],padding = 'SAME')+biases['b1'])
    l1 = tf.nn.max_pool(l1a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    l1 = tf.nn.dropout(l1,p_keep_conv)
    
    #第二组卷积层及池化层，最后dropout部分神经元
    l2a = tf.nn.selu(tf.nn.conv2d(l1,weights['w2'],strides=[1,1,1,1],padding = 'SAME')+biases['b2'])
    l2 = tf.nn.max_pool(l2a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    l2 = tf.nn.dropout(l2,p_keep_conv)
   
    #第三组卷积层及池化层，最后dropout部分神经元
    l3a = tf.nn.selu(tf.nn.conv2d(l2,weights['w3'],strides=[1,1,1,1],padding = 'SAME')+biases['b3'])
    l3 = tf.nn.max_pool(l3a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    l3 = tf.nn.dropout(l3,p_keep_conv)
    l3 = tf.reshape(l3,[-1,w4.get_shape().as_list()[0]])
    
    #全连接层的隐层
    layer_1 = tf.nn.selu(tf.add(tf.matmul(l3, weights['w4']), biases['b4']))
    layer_1 = tf.nn.dropout(layer_1,p_keep_hidden)
    
    #输出层
    l_out = tf.matmul(layer_1, weights['out']) + biases['out']
    
    return l_out
    
    
    
    
    

In [9]:
tf.__version__

'1.6.0'

**隐层的激活函数尝试用RELU，输出层用softmax**

In [10]:
pred = model(x, weights, biases,p_keep_conv,p_keep_hidden) # 前向传播的预测值
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)+ reg_term) # 交叉熵损失函数，参数分别为预测值pred和实际label值y，reduce_mean为求平均loss
optm = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost) # 梯度下降优化器
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) # tf.equal()对比预测值的索引和实际label的索引是否一样，一样返回True，不一样返回False
accr = tf.reduce_mean(tf.cast(corr, tf.float32)) # 将pred即True或False转换为1或0,并对所有的判断结果求均值

init = tf.global_variables_initializer()
print("FUNCTIONS READY")


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

FUNCTIONS READY


In [11]:
# 上面神经网络结构定义好之后，下面定义一些超参数
training_epochs = 100 # 所有样本迭代100次
batch_size = 200 # 每进行一次迭代选择100个样本
test_size = 256
#display_step = 1
# # LAUNCH THE GRAPH
# sess = tf.Session() # 定义一个Session
# sess.run(init) # 在sess里run一下初始化操作

In [17]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(100):
        t1 = time.time()
        avg_cost = 0.
        train_batch = zip(range(0,len(trX),batch_size),
                         range(batch_size,len(trX)+1,batch_size))
        for start ,end in train_batch:
            sess.run(optm,feed_dict={x:trX[start:end],y:trY[start:end],
                                    p_keep_conv:0.8,p_keep_hidden:0.5})
            avg_cost += sess.run(cost, feed_dict={x:trX[start:end],y:trY[start:end],
                                                  p_keep_conv: 0.8,p_keep_hidden: 0.5})/(len(trX)/batch_size)
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        train_acc = sess.run(accr, feed_dict={x:trX[start:end],y:trY[start:end],
                                              p_keep_conv:0.8,p_keep_hidden:0.5})
        test_acc = sess.run(accr, feed_dict={x: teX[test_indices], y: teY[test_indices],
                                             p_keep_conv:0.8,p_keep_hidden:0.5})
        print("Epoch: %03d/%03d cost: %.9f TRAIN ACCURACY: %.3f TEST ACCURACY: %.3f"
              % (i+1, 100, avg_cost, train_acc, test_acc))
        
        t2=time.time()
        print("time:",t2-t1)

Epoch: 001/100 cost: 4.712321234 TRAIN ACCURACY: 0.965 TEST ACCURACY: 0.898
time: 208.72961354255676
Epoch: 002/100 cost: 1.396958885 TRAIN ACCURACY: 0.965 TEST ACCURACY: 0.941
time: 213.8066327571869
Epoch: 003/100 cost: 1.074216613 TRAIN ACCURACY: 0.985 TEST ACCURACY: 0.945
time: 198.86918425559998
Epoch: 004/100 cost: 0.834599558 TRAIN ACCURACY: 0.980 TEST ACCURACY: 0.961
time: 198.89280557632446
Epoch: 005/100 cost: 0.636503658 TRAIN ACCURACY: 0.970 TEST ACCURACY: 0.945
time: 199.29577803611755
Epoch: 006/100 cost: 0.484624510 TRAIN ACCURACY: 0.990 TEST ACCURACY: 0.961
time: 197.94662046432495
Epoch: 007/100 cost: 0.375777249 TRAIN ACCURACY: 0.980 TEST ACCURACY: 0.957
time: 201.15610456466675
Epoch: 008/100 cost: 0.293136587 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.980
time: 199.25197315216064
Epoch: 009/100 cost: 0.236211415 TRAIN ACCURACY: 0.990 TEST ACCURACY: 0.965
time: 197.11867666244507
Epoch: 010/100 cost: 0.191484808 TRAIN ACCURACY: 0.990 TEST ACCURACY: 0.969
time: 196.015265

Epoch: 083/100 cost: 0.060408494 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.969
time: 210.42275738716125
Epoch: 084/100 cost: 0.060326875 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.988
time: 211.099134683609
Epoch: 085/100 cost: 0.061545057 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.988
time: 211.01108503341675
Epoch: 086/100 cost: 0.059381756 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.984
time: 210.6314172744751
Epoch: 087/100 cost: 0.061278177 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.984
time: 209.16399335861206
Epoch: 088/100 cost: 0.059857864 TRAIN ACCURACY: 0.985 TEST ACCURACY: 0.984
time: 211.53392100334167
Epoch: 089/100 cost: 0.059848712 TRAIN ACCURACY: 0.990 TEST ACCURACY: 0.984
time: 204.77457308769226
Epoch: 090/100 cost: 0.062242943 TRAIN ACCURACY: 0.990 TEST ACCURACY: 0.980
time: 200.414067029953
Epoch: 091/100 cost: 0.061534185 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.984
time: 206.11535048484802
Epoch: 092/100 cost: 0.063804752 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.992
time: 215.1133019924

* notebook上只跑了上述的代码，其他的参数尝试在tinymind上跑

**三层卷积情况下**
   * 高斯分布分布初始化参数
   * 三层卷积，卷积核第一层是5,5（想让第一层的感受野大一些），后面两层是3,3，步长1，通道数为16,32,64，准确率在0.98多一点，通道数增大为32，64,128，卷积核全部是3*3准确率提高
   * 最大池化，2*2，步长2， 
   * 全连接用了一个隐层625，
   * 优化器用SGD时，学习率0.01，收敛很慢，后来用了收敛快的RMSPropOptimizer，学习率0.001，收敛很快，最后尝试了AdamOptimizer优化器，学习率0.001，速度很快，AdamOptimizer在tinymind上跑的最后几个结果，准确率波动，需要增大epoch
   
Epoch: 297/300 cost: 0.060344678 TRAIN ACCURACY: 0.995 TEST ACCURACY: 1.000

time: 16.008718490600586

Epoch: 298/300 cost: 0.061144738 TRAIN ACCURACY: 0.990 TEST ACCURACY: 0.977

time: 15.982921838760376

Epoch: 299/300 cost: 0.061210946 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.984

time: 15.960307836532593

Epoch: 300/300 cost: 0.059880949 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.992

time: 15.990186214447021

   * 激活函数用有自归一化的selu，后来换了relu，差别不大
   * L2正则，256，正则参数为1/5000
   * dropout，卷积保留0.8，全连接保留0.5，dropout，都是0.5，差别不大
   
**尝试四层卷积**
  * 加一层卷积使输出变成向量 N,1,1,512，
  * 'w4': init_weights([4,4,256,512]),
   * l4a = tf.nn.relu(tf.nn.conv2d(l3,weights['w4'],strides=[1,1,1,1],padding = 'VALID')+biases['b4'])

在tinymind上跑的结果

Epoch: 296/300 cost: 0.031862083 TRAIN ACCURACY: 0.995 TEST ACCURACY: 1.000

time: 14.93062710762024

Epoch: 297/300 cost: 0.030513806 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.980

time: 14.937577724456787

Epoch: 298/300 cost: 0.034182577 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.988

time: 14.927863597869873

Epoch: 299/300 cost: 0.030758767 TRAIN ACCURACY: 0.990 TEST ACCURACY: 0.988

time: 14.933849334716797

Epoch: 300/300 cost: 0.030601733 TRAIN ACCURACY: 0.995 TEST ACCURACY: 0.992

time: 14.929622888565063